# Rule-based Recommender

This is the most straightforward recommender system. Usually, it is based on rules defined by the business. 

In this case, I will be looking at the following rules:

* Most popular items overall in terms of quantity sold,
* Most popular items by country in terms of quantity sold.

___Items__ are considered to be the unique stock codes._

## Setup

In [13]:
import pandas as pd

from recommender_systems.config import DATA_RAW

In [14]:
data = pd.read_csv(DATA_RAW / "data.csv", encoding="ISO-8859-1")

## Data cleaning

### Investigate data

In [15]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [17]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [18]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Notes:

* The `Quantity` column has negative values. There might be a business reason behind this - for example, back orders, returns, etc. - or it can just be input errors. Since the information on the dataset is limited, I will discard those values.
* The `Description` column has null values which will be discarded. 
* `InvoiceDate` column needs to be converted to a datetime object. 

### Clean data

In [19]:
data_clean = (
    data[data["Quantity"] > 0]
    .dropna(subset=["Description"])
    .assign(
        InvoiceDate=lambda x: pd.to_datetime(x["InvoiceDate"], format="%m/%d/%Y %H:%M"),
        InvoiceMonth=lambda x: x["InvoiceDate"].dt.month,
    )
)

In [20]:
data_clean.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,12


## Retrieve popular items list

In [21]:
def get_top_selling_items(data: pd.DataFrame, by: list[str] | None = None, top_n: int = 10) -> pd.DataFrame:
    """
    Returns the top selling items in the data.

    Parameters
    ----------
    data : pd.DataFrame
        The data to analyze.
    by : list[str] | None, optional
        Additional columns to slice upon, by default None
    top_n : int, optional
        The number of items to return, by default 10

    Returns
    -------
    pd.DataFrame
        The top selling items.
    """
    grain = ["StockCode", "Description"]
    if by:
        grain += by

    temp = data.groupby(grain).agg({"Quantity": "sum"}).sort_values(["Quantity"], ascending=False)

    if by:
        return temp.groupby(level=by).head(top_n).reset_index(drop=False)

    return temp.head(top_n)

### globally

In [22]:
# Top 10 stock codes by number of transactions
get_top_selling_items(data_clean)

,,Quantity
StockCode,Description,
23843,"PAPER CRAFT , LITTLE BIRDIE",80995
23166,MEDIUM CERAMIC TOP STORAGE JAR,78033
84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,55047
85099B,JUMBO BAG RED RETROSPOT,48478
85123A,WHITE HANGING HEART T-LIGHT HOLDER,37603
22197,POPCORN HOLDER,36761
84879,ASSORTED COLOUR BIRD ORNAMENT,36461
21212,PACK OF 72 RETROSPOT CAKE CASES,36419
23084,RABBIT NIGHT LIGHT,30788


### by country

In [23]:
# Top 10 stock codes by country in terms of quantity sold
get_top_selling_items(data_clean, by=["Country"])

,StockCode,Description,Country,Quantity
0,23843,"PAPER CRAFT , LITTLE BIRDIE",United Kingdom,80995
1,23166,MEDIUM CERAMIC TOP STORAGE JAR,United Kingdom,77036
2,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,United Kingdom,49526
3,85099B,JUMBO BAG RED RETROSPOT,United Kingdom,44268
4,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom,35452
...,...,...,...,...
374,22699,ROSES REGENCY TEACUP AND SAUCER,Bahrain,6
375,22363,GLASS JAR MARMALADE,Saudi Arabia,6
376,22362,GLASS JAR PEACOCK BATH SALTS,Saudi Arabia,6
377,22361,GLASS JAR DAISY FRESH COTTON WOOL,Saudi Arabia,6


### by country and month sold

In [24]:
# Top 10 stock codes by country and month in terms of quantity sold
get_top_selling_items(data_clean, by=["Country", "InvoiceMonth"])

,StockCode,Description,Country,InvoiceMonth,Quantity
0,23843,"PAPER CRAFT , LITTLE BIRDIE",United Kingdom,12,80995
1,23166,MEDIUM CERAMIC TOP STORAGE JAR,United Kingdom,1,74215
2,84826,ASSTD DESIGN 3D PAPER STICKERS,United Kingdom,11,12551
3,22197,POPCORN HOLDER,United Kingdom,11,12038
4,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,United Kingdom,4,9567
...,...,...,...,...,...
2772,23194,GYMKHANA TREASURE BOOK BOX,Japan,3,1
2773,23177,TREASURE ISLAND BOOK BOX,Japan,3,1
2774,22424,ENAMEL BREAD BIN CREAM,European Community,4,1
2775,22758,LARGE PURPLE BABUSHKA NOTEBOOK,Netherlands,7,1
